This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [45]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# This step has been commented out to protect my privacy.
# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [46]:
# First, import the relevant modules
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [47]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
params = dict(start_date='2014-04-29', end_date='2014-04-30', api_key=API_KEY)
res = requests.get(url, params)
my_json = res.json()

In [48]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(my_json, indent=1))

{
 "dataset_data": {
  "limit": null,
  "transform": null,
  "column_index": null,
  "column_names": [
   "Date",
   "Open",
   "High",
   "Low",
   "Close",
   "Change",
   "Traded Volume",
   "Turnover",
   "Last Price of the Day",
   "Daily Traded Units",
   "Daily Turnover"
  ],
  "start_date": "2014-04-29",
  "end_date": "2014-04-30",
  "frequency": "daily",
  "data": [
   [
    "2014-04-30",
    21.9,
    22.04,
    21.84,
    21.93,
    null,
    34827.0,
    765135.0,
    null,
    null,
    null
   ],
   [
    "2014-04-29",
    21.94,
    22.29,
    21.94,
    22.13,
    null,
    41050.0,
    909259.0,
    null,
    null,
    null
   ]
  ],
  "collapse": null,
  "order": null
 }
}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [49]:
# 1 & 2. In this cell, we will collect the data and make it a dictionary.

# Specity our desired dates.
our_start_date = '2017-01-01'
our_end_date = '2017-12-31'

# Package them, along with our API key, as parameters to use in the API request.
params = dict(start_date=our_start_date, end_date=our_end_date, api_key=API_KEY)

# Make the API request with the URL we saved above and our current parameters.
res = requests.get(url, params)

# Get the JSON we need.
my_json = res.json()

# Dictify it.
my_dict = dict(my_json)

In [50]:
# 3. In this cell, we find the highest and lowest opening prices of 2017.

# Make blank lists to store all opening prices in chron order.
opening_prices = []

# Iterate through the object and store the values in the list.
# To prevent missing values, and also to preserve the order & 
# length of the list, we'll impute the 'None' values by replacing
# them with the next valid day's opening price.
# The "opening price" is in index position 1 of the dict value's list.
for i in my_dict['dataset_data']['data'] :
    if i[1] :
        opening_prices.append(i[1])
    else :
        opening_prices.append(opening_prices[-1])

# Find the max and min.
highest_opening_price = max(opening_prices)
lowest_opening_price = min(opening_prices)

print("The highest opening price in 2017 was "+str(highest_opening_price))
print("The lowest opening price in 2017 was "+str(lowest_opening_price))

The highest opening price in 2017 was 53.11
The lowest opening price in 2017 was 34.0


In [51]:
# 4. In this cell, we find the largest single day change in 2017.

# We must compare the High values to the Low values, so let's make
# lists for them.
high_values = []
low_values = []

# Iterate over the object to input the values.
# The daily high is in index position 2.
# The daily low is in index position 3.
# Since we haven't been asked to say which day the value occurred,
# we can impute using the same method as above (replacing None values
# with the previous non-None day's values).
for i in my_dict['dataset_data']['data'] :
    if i[2] :
        high_values.append(i[2])
    else :
        high_values.append(high_values[-1])
    if i[3] :
        low_values.append(i[3])
    else:
        low_values.append(low_values[-1])

# Find the difference between each day in order, storing them in a list.
# Since we can't use numpy, we'll have to iterate again.
differences = []
for h, l in zip(high_values, low_values) :
    difference = h - l
    differences.append(difference)

# To find the highest change, we'll make a list of the absolute values
# of the differences and use that list to find the location of the
# biggest difference in the +- list, then print that value as the answer.
abs_differences = []
for i in differences :
    abs_differences.append(abs(i))

# Find the index of the biggest value.
biggest_change_index = abs_differences.index(max(abs_differences))

# Store the corresponding value in a variable.
biggest_change = differences[biggest_change_index]

# Round to 2 decimal places, like the original data.
biggest_change_rounded = round(biggest_change, 2)

print("The highest single-day change in 2017 was "+str(biggest_change_rounded))

The highest single-day change in 2017 was 2.81


In [52]:
# 5. In this cell, we'll find the largest change between any two days as
# calculated by the difference of the closing prices.

# Let's make a list to store those prices.
closing_prices = []

# Now, get the values.
# Looping and inputing as before.
# The closing prices are at index position 4.
for i in my_dict['dataset_data']['data'] :
    if i[4] :
        closing_prices.append(i[4])
    else :
        closing_prices.append(closing_prices[-1])

# Now we'll make a new list of the differences between consecutive values.
closing_differences = []
for i in range(len(closing_prices)) :
    if i < len(closing_prices) :
        closing_differences.append(closing_prices[i] - closing_prices[i-1])

# Again with the absolute values...
abs_closing_differences = []
for i in closing_differences :
    abs_closing_differences.append(abs(i))

biggest_value_index = abs_closing_differences.index(max(abs_closing_differences))

biggest_value = closing_differences[biggest_value_index]

biggest_value_rounded = round(biggest_value, 2)

print("The largest change between any two days in 2017 was "+str(biggest_value_rounded))

The largest change between any two days in 2017 was 15.96


In [53]:
# 6. In this cell, we will calculate the average daily trading volume for the year.
trading_volumes = []

# Get 'em.
# This time we won't impute the missing values. We'll just drop them.
# This will prevent skewing the mean, and order/length could not conceivably
# matter for any future uses of these lists.
# Trading volumes are in index position 6.
for i in my_dict['dataset_data']['data'] :
    if i[4] :
        trading_volumes.append(i[6])

# Now we just average them and round for readability.
trading_volume_avg = sum(trading_volumes) / len(trading_volumes)
trading_volume_avg_rounded = round(trading_volume_avg, 2)

print('The average daily trading volume in 2017 was '+str(trading_volume_avg_rounded))

The average daily trading volume in 2017 was 89124.34


In [54]:
# 7. In this cell, we will find the median daily trading volume.

# We'll just order the list we created in the previous cell,
# then either choose the middle value or average the two middle
# values.

trading_volumes_sorted = sorted(trading_volumes)
length = len(trading_volumes_sorted)

# Determing if the number of values is even or odd, then calculate
# the median accordingly.
if length % 2 == 0 :
    upper_index = int((length / 2) + 0.5)
    lower_index = int((length / 2) - 0.5)
    median_value = (trading_volumes_sorted[upper_index] - trading_volumes_sorted[lower_index]) / 2
else :
    median = trading_volumes_sorted[int(length/2)]

print("The median daily trading volume in 2017 was "+str(median))

The median daily trading volume in 2017 was 76286.0
